# MATH 405/607 

# Numerical Methods for Differential Equations

[[Instructor: Christoph Ortner]](http://www.math.ubc.ca/~ortner/)  [[course page]](https://github.com/cortner/math405_2022)


## Solving Nonlinear Systems

* Iterative solution of equations
* Bisection method
* Newton method
* Newton method for systems
* Fixed point iterations

In [ ]:
include("math405.jl")

### Literature

* E. Süli and D. Mayer, An Introduction to Numerical Analysis, Ch. 1, 4
* https://fncbook.github.io/fnc/nonlineqn/overview.html

Further references see end of lecture.

## The Rootfinding Problem

Given an interval $I \subset \mathbb{R}$ and $f \in C(I; \mathbb{R})$ find $x \in I$ such that $f(x) = 0$. We call such an $x$ a *root*.

In the first part of this lecture we will explore methods to solve this problem and use it also to illustrate the concept of *convergence rates*. In the second part of  the lecture we will extend this to nonlinear systems where $f : \Omega \to \mathbb{R}^N$ with $\Omega \subset \mathbb{R}^N$.

In the following in addition to $f$ being continuous we will implicitly assume any regularity required for the argument / calculation that we are performing. 

### Example

This is taken from [fnc](https://fncbook.github.io/fnc/nonlineqn/demos/roots-bessel.html)

In the theory of [vibrations of a circular drum](https://en.wikipedia.org/wiki/Vibrations_of_a_circular_membrane), the displacement of the drumhead can be expressed in terms of pure harmonic modes
$$ 
    J_m(\omega_{k,m} r) \cos(m \theta) \cos(c \omega_{k,m} t),
$$
where the $J_m$ are the *Bessel functions of the first kind*. The *resonant frequencies* $\omega_{k,m}$ are the positive roots of $J_m$: 
$$ 
    J_m(\omega_{k,m}) = 0
$$

In [ ]:
using SpecialFunctions 
plot(x->besselj(3,x), 0, 20, lw=3, label="", grid=:xy, size=(400,200))

In [ ]:
using NLsolve # or use Roots.jl instead!
guess = [6.0, 10.0, 13.0, 16.0, 18.0]
R = [ nlsolve(x->besselj(3,x[1]), [x0]).zero[1] for x0 in guess  ]

In [ ]:
plot(x->besselj(3,x), 0, 20, lw=3, label="", grid=:xy, size=(500,300))
scatter!( guess, besselj.(3, guess), m=:square, ms=7, label = "guesses" )
scatter!( R, besselj.(3, R), ms=7, label = "roots" )

Most roots (green) are close to the initial guesses (red) but one is quite far away, and in fact because of that we didn't find one of the roots we were looking for. This is a common challenge in the solution of nonlinear equations or systems.

In [ ]:
?nlsolve

So we need to go to the documentational of [NLsolve.jl](https://github.com/JuliaNLSolvers/NLsolve.jl) ...

... with a bit of digging we can find out that we have used a **Newton-Trust Region Method**. The trust region aspect is a globalisation strategy which we won't study in this class. The core search functionality is provided by the Newton method: 

### Newton's Method

Let $f : I \to \mathbb{R}$ and suppose that there is a root $r \in {\rm int}(I)$ and that we have a guess $x_n$ for $r$ which is "close" to $r$. 

We can expand $f$ around $x$ 

$$ 
f(x+h) = f(x) + f'(x) h + O(h^2)
$$

If we choose $h$ such that $f(x) + f'(x) h = 0$ then, formally, $f(x+h) = O(h^2)$. That is, if the step $h$ is small then $x+h$ will be close to a root! 


This suggests the following *Newton Iteration*: 
$$ 
    x_{n+1} := x_n - f(x_n) / f'(x_n)
$$

Or alternatively we can derive the method geometrically, but this makes it more difficult to understand the convergence.

![newtonurl](https://upload.wikimedia.org/wikipedia/commons/e/e0/NewtonIteration_Ani.gif)

<!-- 
Creative Commons Attribution-Share Alike 3.0 (CC BY-SA 3.0)
http://creativecommons.org/licenses/by-sa/3.0/
-->

In [ ]:
f = x -> besselj(3, x)
df = x -> ForwardDiff.derivative(f, x)
x = guess[1]; iter = [x]
p = plot(f, 5.5, 7, lw=3, label=L"J_3", grid=:xy, size=(800,300), legend=:outertopright)
scatter!([x], [f(x)], ms=6, label="|f(x_0)| = $(abs(f(x)))")
for n = 2:5.5
    x -= f(x) / df(x) ; push!(iter, x)
    scatter!([x], [f(x)], ms=6, label="|f(x_$n)| = $(abs(f(x)))")
end
p


In [ ]:
f = x -> besselj(3, x)
df = x -> ForwardDiff.derivative(f, x)
x = guess[1]
p = plot(abs ∘ f, 5.5, 7, lw=2, label=L"J_3", grid=:xy, size=(700,300), 
          legend=:outertopright, yaxis = :log)
scatter!([x], [abs(f(x))], ms=6, label="|f(x_0)| = $(abs(f(x)))")
for n = 2:5
    x -= f(x) / df(x) 
    scatter!([x], [abs(f(x))], ms=6, label="|f(x_$n)| = $(abs(f(x)))")
end
p


In [ ]:
# Look at this more closely: 
pretty_table( [iter f.(iter) abs.(iter .- iter[end])],  ["Iterates", "f(x)", "|x-r|"],
               formatters = (v, i, j) -> (@sprintf("%1.20f", v))  )

### Newton Method: Local Convergence

**Theorem:** Let $f \in C^2(r - \delta, r + \delta)$ for some $\delta > 0$ and let $f(r) = 0, f'(r) \neq 0$. Then there exists $\epsilon > 0$ such that for all $x_0 \in (r - \epsilon, r + \epsilon)$ the Newton iteration converges $x_n \to r$ as $n \to \infty$. Moreover, the convergence is quadratic, i.e., there exists a constant $C$ such that 
$$ 
    |x_{n+1} - r| \leq C |x_n - r|^2
$$

**Proof:** see tablet/board notes. 

### Newton Method: Global Convergence

The example with the roots of the bessel function shows that nonlinear iterations can be unpredictable. Newton's method is a paradigm example of this. 

In [ ]:
f = x -> x^3 - 2x + 2
df = x -> 3 * x^2 - 2 

function plot_iter(x)
    xn = x - f(x) / df(x)
    plot(f, -0.5, 1.5, lw=2, label = "", size=(600, 400))
    plot!([x, xn], [f(x), 0.0], lw=3, ls=:dash, c=:black, label = "tangent")
    scatter!([x], [f(x)], ms=8, label = "current")
    return scatter!([xn], [f(xn)], ms=8, label = "new"), xn 
end 

with_logger(NullLogger()) do    # supress some unhelpful info...
    xn = 0.0
    anim = @animate for n = 1:4; plt, xn = plot_iter(xn); plt; end
    gif(anim, fps=1);
end

**Easter egg:** But Newton's strange behaviour is much more fun to explore in $\mathbb{C}$. 
- Consider $z^3 = 1$ in $\mathbb{C}$. 
- The solutions are the roots of unity. 
- For each $z_0 \in \mathbb{C}$ apply Newton's method
- check to which root it converges. Color accordingly.

In [ ]:
using Images
function paint(x, y; maxn = 30)
    f = z -> z^3 - 1; df = z -> 3*z^2
    roots = [ exp(im * 2 * π * n/3) for n in 0:2 ]
    n = 0; z = x + im*y
    while abs(f(z)) > 1e-10 && n < maxn
        z -= f(z) /  df(z)
        n += 1
    end 
    i = findmin(abs.(roots .- z))[2]
    val = 1 - n/maxn
    return RGB(val*(i==1), val*(i==2), val*(i==3))
end
img = [ paint(x, y) for x in range(-1,1,length=500), y in range(-1,1, length=500) ];

In [ ]:
img

This is called a Newton-type fractal. For more on this see the vast literature on Fractals. For a much better implementation and more examples see [Fatou.jl](https://github.com/chakravala/Fatou.jl)

## Bisection 

Can we develop a scheme that is less chaotic? Yes, if we use additional information about a potential root:

**Lemma [Bolzano]:** Let $f \in C([a, b]), f(a) f(b) < 0$, then $\exists$ a root $r \in (a, b)$.

In [ ]:
f = x -> besselj(3, x)
plot(f, 0, 10, lw=3, label="", size=(400, 200))
a, b = 5.0, 7.5 
scatter!([a,b], [f(a), f(b)], ms=8, label = "f(a), f(b)")

Idea: shrink the interval until $|b-a| < {\rm TOL}$

In [ ]:
function bisection(f, a, b, tol)
    while abs(b-a) > tol 
        c = 0.5 * (a+b)
        sign(f(c)) == sign(f(a)) ? a = c : b = c
    end             
    return 0.5*(a+b)
end

In [ ]:
x = bisection(f, a, b, 1e-4)
println("x = ", x)
println("f(x) = ", f(x))
println("error = ", abs(x - R[1]))

In [ ]:
# Let's look at the bisection method in action
with_logger(NullLogger()) do    # supress some unhelpful info...
    a, b = 5.0, 7.5 
    plt = plot(f, 0, 10, lw=3, label="", size=(500, 200))
    anim = @animate for n = 0:5 
        plt = scatter!([a,b], [f(a), f(b)], ms=8, label = "", title = "|a-b| = $(abs(a-b))")
        c = 0.5 * (a+b); sign(f(c)) == sign(f(a)) ? a = c : b = c
        plt 
    end             
    gif(anim, fps=1)
end


"Clean" Bisection algorithm:
```
While abs(b-a) > tol 
    c = 0.5 * (a+b) 
    If f(c) == 0
       return [a, c]
    Elseif sign(f(c)) == sign(f(a))
       a = c 
    Else
       b = c
    End
End
return [a, b]    
```



**Theorem:** If the input is admissible, then the bisection algorithm terminates after finitely many steps with an interval $[a, b]$ containing a root such that $|b - a| \leq {\rm tol}$.

**Proof:** After each iteration $|b-a|$ is halved. Therefore after $O(\log \tau)$ iterations the algorithm terminates. At each iteration we have $f(a)f(b) \leq 0$, hence the output contains a root.

### Convergence of the bisection algorithm

Let $[a_n, b_n]$ be the interval at the $n$-th iteration of the bisection algorithm, then 
$$
    |a_n - b_n| = 2^{-n} |a_0 - b_0|
$$

and in particular 
$$
    |r - 0.5(a_n+b_n)| \leq 2^{-n-1} |a_0 - b_0|.
$$

These are instances of *linear convergence*. At each iteration the number of correct digits increases by a constant.

COST: $\epsilon(n) \approx c 2^{-n}$ then $n(\epsilon) \approx \log_2(\epsilon)$

## Convergence Rates 

**Definition:** Let $x_n \in \mathbb{F}^N$ and let $d$ be a metric defined on $\mathbb{F}^N$.
1. We say that a sequence $x_n \to x$ (Q-)linearly with *rate of convergence* $\mu \in (0, 1)$ if 
$$ 
    \limsup_{n\to\infty} \frac{d(x_{n+1}, x)}{d(x_n, x)} = \mu.
$$

2. We say that $x_n \to x$ with order $q > 1$ if $x_n \to x$ and 
$$ 
    \limsup_{n \to \infty} \frac{d(x_{n+1}, x_n)}{d(x_n, x_{n-1})^q} =: M \in (0, \infty).
$$
If $q = 2$ then we call this *quadratic convergence* 

3. We say that $x_n \to x$ R-linearly, R-quadratically, etc if there exists a sequence $\epsilon_n \to 0$ R-linearly, R-quadratically, etc, such that $d(x_n, x) \leq \epsilon_n$. 

The last definition is to cover cases where the convergence is not very uniform.

## Nonlinear Systems

We now turn towards nonlinear systems. Throughout the remainder of this lecture, we let $D \subset \mathbb{R}^N$ be open and simply connected (a *domain*) and $f \in C(D; \mathbb{R}^M)$ but as before we assume higher regularity as needed. 

When we write $f \in C^p(D; \mathbb{R}^M)$ we will always mean differentiability in the sense of Frechet. If you are unfamiliar or uncomfortable with this definition, then you can look it up in any multi-variate analysis textbook or lecture notes or if need be on [wikipedia](https://en.wikipedia.org/wiki/Fréchet_derivative). However, for this course a deeper understanding should not should not be required and we can work with a simple and intuitive definition of differentiation, which essentially says that $f$ has a (multi-variate) Taylor expansion: 

**Definition:** we say that $f$ is differentiable at a point $x_0 \in D$ if there exists a matrix $\partial f(x) \in \mathbb{R}^{M \times N}$ such that for all $x \in D$, 

$$ 
    f(x) = f(x_0) + \partial f(x_0) (x-x_0) + \epsilon(x_0; x-x_0)
$$

where $\| \epsilon(x_0, x-x_0) \| = o(\| x - x_0 \|)$ (i.e. the remainder may be neglected as $x \to x_0$. The matrix $\partial f(x_0)$ is called the Jacobi matrix. (the matrix of partial derivatives)

We will normally have at least $f \in C^2$ and in this case we can replace $\epsilon$ with $O(\|x - x_0\|^2)$. We will assume this from now on.

### Newton's method for systems 

That definition also happens to be the derivation of Newton's method: if $f \in C^2$ then 

$$
    f(x) = f(x_0) + \partial f(x_0) (x-x_0) + O(\|x - x_0\|^2)
$$

Given an iterate $x_n$ we choose the next iterate $x_{n+1}$ such that 

$$ 
    f(x_n) + \partial f(x_n) (x_{n+1} - x_n) = 0
$$

This requires the solution of an $N \times N$ linear system.

If $x_n$ is close to a root $r$ then we will  have that $\| f(x_{n+1})  \| \lesssim C \| x_n - x \|^2$ and therefore we can expect that $\| x_{n+1} - r \| \leq C \| x_n - r \|^2$. 

Newton iteration: 
$$ 
    x_{n+1} = x_n - \partial f(x_n)^{-1} f(x_n)
$$

**Theorem:** Suppose that $f \in C^2(D; \mathbb{R}^N)$, $r \in D$, $f(r) = 0$ and $\partial f(r)$ is non-singular, then there exists $\epsilon > 0$ such that for $\|x_0 - r\| \leq \epsilon$ the Newton-iteration is well-defined and $x_n \to r$ quadratically.

**Proof:** see board/tablet/recording.

In [ ]:
function newton(f, x0, tol, df = x -> ForwardDiff.jacobian(f, x))
    while norm(f(x)) > tol 
        x = x - df(x) \ f(x)
    end 
    return x
end

In [ ]:
function newtonhist(f, x0, niter, df = x -> ForwardDiff.jacobian(f, x))
    hist = [x0]; x = x0 
    for n = 1:niter
        x -= df(x) \ f(x) 
        push!(hist, x)
    end
    return hist 
end

**Example:** two simultaneous equations defining the intersection of 2 ellipsi
$$
x_1^2 + x_2^2 = 1, \qquad \qquad 
    5 x_1^2 + 21 x_2^2 = 9
$$ 

In [ ]:
tt = range(0, 2*pi, length = 300)
function _plot_sys()
    plot( cos.(tt), sin.(tt), lw=3, label = "Eqn 1", size = (350,250), legend=:outertopright)
    plot!( 3/√5 * cos.(tt), 3/√21 * sin.(tt), lw=3, label = "Eqn 2")
end 
_plot_sys()

In [ ]:
f = x -> [ x[1]^2+x[2]^2 - 1, 5*x[1]^2+21*x[2]^2 - 9 ]
X = newtonhist(f, [0.3,0.3], 7)
_plot_sys()
scatter!( [x[1] for x in X], [x[2] for x in X], ms=6, label = "")

In [ ]:
pretty_table( [[x[1] for x in X] [x[2] for x in X] norm.(f.(X))], 
               ["x1", "x2", "|f(x)|"], 
              formatters = (v, i, j) -> (@sprintf("%1.20f", v))  )

Main message is : the theory and practise of Newton's method for system is essentally the same as for scalar problems.

- for non-singular systems the local convergence is very rapid (quadratic) 
- global convergence is not guaranteed, and difficult to predict.
- For many practical problems it is difficult to get the Jacobi matrix. 

## Fixed Point Iterations

For many nonlinear problems there will be problem-specific iteration strategies that might include some ideas of Newton's method (or not...) Most of these iterations are best thought of as fixed-point iterations: 
- construct a *fixed point map* $T : \mathbb{R}^N \to \mathbb{R}^N$, or $T : D \to D$ such that $f(x) = 0$ iff. $T(x) = x$. 
- iterate $x_{n+1} = T(x_n)$.

We will treat this topic only theoretically and by means of a few examples.

<!-- Trivial construction: $T(x) = x - \alpha f(x)$ -->

**Example 1:** 
$$ 
    T(x) = x + \alpha f(x)
$$
then clearly $f(x) = 0$ iff $T(x) = x$. This can be thought of as a discretisation of the ODE $\dot{x} = f(x)$. If $r$ is a *stable* equilibrium then for sufficiently small $\alpha$ and $x_0$ sufficiently close to  $r$ we have $x_n \to r$.

**Example 2:** The Newton iteration 

$$
  x_{n+1} = x_n - \partial f(x_n)^{-1} f(x_n)
$$

is a fixed point iteration $x_{n+1} = T(x_n)$ with operator

$$ 
T(x) = x - \partial f(x)^{-1} f(x)
$$


**Example 3:** Many nonlinear PDEs can be rewritten in a fixed-point form, e.g., consider 
$$
   \nabla \cdot \big( a(x, u(x)) \nabla u(x) \big) = b(x, u(x))
$$

Then we can define a fixed point operator by : to get the mapping $u \mapsto Tu$ solve the linear equation
$$ 
   \nabla \cdot \big( a(x, u(x)) \nabla Tu(x) \big) =  b(x, u(x))
$$
for $Tu$. $T(u)$

### Fixed Point Theorems

There are numerous results that will establish the existence of fixed points, the convergence of fixed point iterations, or the acceleration of fixed point iterations. Here we mention just two elementary results of this kind: 

**Theorem:** (Brouwer's fixed point theorem) Let $D \subset \mathbb{R}^N$ be non-empty, closed, bounded, and convex and $T : D \to D$  continuous, then there exists $x_* \in D$ s.t. $T(x_*) = x_*$.

This is quite abstract and does not say much about iterations and practical computations. One usually aims for stronger results.

**Theorem:** (Contraction mapping theorem, Banach's fixed point theorem) 
Suppose that $D \subset \mathbb{R}^N$ is closed, $T : D \to D$ is a contraction, i.e., $\|T(x) - T(y)\| \leq \eta \|x - y \|$ where $\eta \in (0, 1)$, then there exists a unique fixed point in $D$ and the iteration $x_{n+1} = T(x_n)$ converges linearly with rate $\eta$.

**Proof:** see any introductory functional analysis textbook, or Mayers & Süli, §4.2.

<br> 

In fact, this is one way to prove convergence of Newton's method.

### Further Topics 

* Damped Newton method: consider steps $x_{n+1} = x_n - \alpha_n \partial f(x_n)^{-1} f(x_n)$ and choose $\alpha_n$ to control the convergence, see `NLsolve.jl`
* Line search methods: means to determine step-lengths $\alpha_n$, see `NLsolve.jl`
* Trust region methods: alternative globalisation strategy, see `NLsolve.jl`
* Quasi-Newton methods: Replace $\partial f(x_n)^{-1}$ with a low-rank approximation which is estimated from past iterates 
* Jacobian-free methods: Replace $\partial f(x_n) \cdot u$ with $(f(x_n + \delta u) - f(x_n))/\delta$ and use iterative linear solvers

### Further reading 
* C. T. Kelley, Iterative Methods for Linear and Nonlinear Equations [pdf](https://archive.siam.org/books/textbooks/fr16_book.pdf)
* J. M. Ortega and W. C. Rheinboldt. Iterative Solution of Nonlinear Equations in Several Variables. Elsevier, 2014.
* C. T. Kelley, Solving Nonlinear Equations with Newton's Method
* P. Deuflhard, Newton Methods for Nonlinear Problems - Affine Invariance and Adaptive Algorithms
* J. Nocedal and S. Wright, Numerical Optimization (I will try to fit in a class on nonlinear optimization later in the course)

In Julia see 
* [NLSolve.jl](https://github.com/JuliaNLSolvers/NLsolve.jl) : nonlinear systems
* [Roots.jl](https://github.com/JuliaMath/Roots.jl) : solving scalar nonlinear equations
* [SIAMFANLEquations.jl](https://github.com/ctkelley/SIAMFANLEquations.jl) : Kelley's codes